**Conjugate Gradient**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


6. Conjugate Gradient
  
    Matrix Requirement: Square, SPD

In [3]:
import numpy as np
from scipy.sparse import csr_matrix

def conjugate_gradient(A, b, max_iter=1000, tol=1e-10):
    # Keep A as sparse for efficiency
    n = A.shape[0]
    x = np.zeros(n)  # initial guess
    r = b - A @ x
    p = r.copy()
    rs_old = np.dot(r, r)

    for it in range(max_iter):
        Ap = A @ p
        alpha = rs_old / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rs_new = np.dot(r, r)

        if np.sqrt(rs_new) < tol:
            return x, it+1, np.sqrt(rs_new)

        p = r + (rs_new / rs_old) * p
        rs_old = rs_new

    return x, max_iter, np.sqrt(rs_new)

# Example usage
b = np.random.rand(A_sparse.shape[0])  # RHS vector
x, iterations, final_residual = conjugate_gradient(A_sparse, b)

print("CG solution vector (first 5 entries):", x[:5])
print("Iterations:", iterations)
print("Final residual norm:", final_residual)

CG solution vector (first 5 entries): [ 1.87457240e-08 -7.62040728e-09  9.14045296e-01  2.12176959e-08
 -6.01523343e-09]
Iterations: 562
Final residual norm: 8.739837859191726e-11
